In [1]:
# Import libraries 
import extract_eye_features
import dill
import scipy
import os
import numpy as np
import sys 

sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import matplotlib.pyplot as plt
import BLNK_eye_extraction_pipeline

Loading DLC 3.0.0rc8...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [ ]:
# Define the path to the folder containing the videos and other files 
path_to_video_folder: str = "/Users/zacharykelly/Desktop/BLNK_1008"

# Generate paths to all of the videos in said folder 
video_paths: list[str] = [os.path.join(path_to_video_folder, filename)
                          for filename in os.listdir(path_to_video_folder)
                          if '.avi' in filename
                         ]

In [ ]:
# Generate output folder 
output_folder_path: str = "/Users/zacharykelly/Aguirre-Brainard Lab Dropbox/Zachary Kelly/BLNK_analysis/PuffLight/lightLevel/BLNK_1008"
if(not os.path.exists(output_folder_path)):
    os.mkdir(output_folder_path)

In [7]:
# Generate a temp output directory 
temp_dir_path: str = './temp_blnk_pipeline'
if(not os.path.exists(temp_dir_path)):
    os.mkdir(temp_dir_path)

# Define the crop box and target size 
t, b, l, r = 140, 275, 190, 425
crop_box: tuple = (t, b, l, r)
target_size: tuple = (480, 640)

# Iterate over the videos and output them to the target directory 
for video_num, video_path in enumerate(video_paths):
    print(f"Processing video: {video_num}/{len(video_paths)}", flush=True)
    BLNK_eye_extraction_pipeline.predict_eye_features(video_path, output_folder_path, crop_box, target_size, temp_dir_path)



    

Processing video: 0/26
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santini_eyelid_detectionJul182025-trainset99shuffle1/train/snapshot-best-220.pt
Starting to analyze temp_blnk_pipeline/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-002_side-R.avi
Video metadata: 
  Overall # of frames:    5760
  Duration of video [s]:  32.00
  fps:                    180.0
  resolution:             w=640, h=480

Running pose prediction with batch size 8


100%|██████████| 5760/5760 [03:15<00:00, 29.40it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmped48t5j1/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-002_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmped48t5j1/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-002_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmped48t5j1/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-002_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']


100%|██████████| 5760/5760 [04:18<00:00, 22.25it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpv0_fysgq/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-002_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpv0_fysgq/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-002_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpv0_fysgq/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.15_trial-002_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5']


...

100%|██████████| 5760/5760 [03:13<00:00, 29.79it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpp__6lpoi/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.04_trial-013_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpp__6lpoi/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.04_trial-013_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpp__6lpoi/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.04_trial-013_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']


100%|██████████| 5760/5760 [04:11<00:00, 22.87it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpqc15_gso/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.04_trial-013_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpqc15_gso/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.04_trial-013_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpqc15_gso/temp_BLNK_1006_lightLevel_direction-LightFlux_sequence-1_contrast-0.04_trial-013_side-RDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_110.h5']


/opt/anaconda3/envs/pylids/lib/python3.10/site-packages/numpy/polynomial/polynomial.py:1362: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)
